# Flow and task execution

In [4]:
import time
from prefect import task, flow

## Task runners
Task runners are responsible for running Prefect tasks within a flow. Each flow has a task runner associated with it. 

Depending on the task runner you use, the tasks within your flow can run sequentially, concurrently, or in parallel.

You can even configure task runners to use distributed execution infrastructure such as a Dask cluster.

To make sure that the tasks within your flow can run concurrently or in parallel, add .submit() to your task run. This method will return a PrefectFuture instead of a Python object.

A PrefectFuture is an object that provides access to a computation happening in a task runner

## Concurrent execution
by default Prefect flows use the ConcurrentTaskRunner for non-blocking, concurrent execution of tasks.

In [2]:


@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()


21:24:37.913 | INFO    | prefect.engine - Created flow run 'phenomenal-spider' for flow 'my-flow'
21:24:38.030 | INFO    | Flow run 'phenomenal-spider' - Created task run 'print_values-0bb9a2c3-0' for task 'print_values'
21:24:38.030 | INFO    | Flow run 'phenomenal-spider' - Submitted task run 'print_values-0bb9a2c3-0' for execution.
21:24:38.043 | INFO    | Flow run 'phenomenal-spider' - Created task run 'print_values-0bb9a2c3-1' for task 'print_values'
21:24:38.044 | INFO    | Flow run 'phenomenal-spider' - Submitted task run 'print_values-0bb9a2c3-1' for execution.


21:24:43.134 | INFO    | Task run 'print_values-0bb9a2c3-1' - Finished in state Completed()


21:24:45.655 | INFO    | Task run 'print_values-0bb9a2c3-0' - Finished in state Completed()
21:24:45.674 | INFO    | Flow run 'phenomenal-spider' - Finished in state Completed('All states completed.')


## Sequential execution

In [3]:
from prefect.task_runners import SequentialTaskRunner

@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(task_runner=SequentialTaskRunner())
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()

/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/tasks.py:198: UserWarning: A task named 'print_values' and defined at '/var/folders/v2/kxvx63cs0nj24qk2_m4cyb0r0000gn/T/ipykernel_26998/3164186700.py:3' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/flows.py:201: UserWarning: A flow named 'my-flow' and defined at '/var/folders/v2/kxvx63cs0nj24qk2_m4cyb0r0000gn/T/ipykernel_26998/3164186700.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
21:27:32.812 | INFO    | prefect.engine - Created flow run 'bizarre-jackdaw' for flow 'my-flow'
21:27:32.914 | INFO    | Flow run 'bizarre-jackdaw' - Created task run 'print_values-0bb9a2c3-0' for task 'print_values'
21:27:32.914

21:27:40.522 | INFO    | Task run 'print_values-0bb9a2c3-0' - Finished in state Completed()
21:27:40.538 | INFO    | Flow run 'bizarre-jackdaw' - Created task run 'print_values-0bb9a2c3-1' for task 'print_values'
21:27:40.538 | INFO    | Flow run 'bizarre-jackdaw' - Executing 'print_values-0bb9a2c3-1' immediately...


21:27:45.615 | INFO    | Task run 'print_values-0bb9a2c3-1' - Finished in state Completed()
21:27:45.633 | INFO    | Flow run 'bizarre-jackdaw' - Finished in state Completed('All states completed.')


## Parallel execution

In [5]:
from prefect_dask.task_runners import DaskTaskRunner

You can also run tasks using parallel or distributed execution by using the Dask or Ray task runners

In [7]:
@task
def print_values3(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(task_runner=DaskTaskRunner())
def my_flow3():
    print_values3.submit(["AAAA"] * 15)
    print_values3.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow3()

21:35:16.082 | INFO    | prefect.engine - Created flow run 'vigilant-bee' for flow 'my-flow3'
21:35:16.083 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`
2022-10-24 21:35:17,027 - distributed.worker - WARNING - Mismatched versions found

+---------+---------------------------------------------+-----------+---------+
| Package | Worker-09cf5d7a-dd50-4c70-b78f-70f26fabb1f0 | Scheduler | Workers |
+---------+---------------------------------------------+-----------+---------+
| tornado | 6.1                                         | 6.2       | 6.1     |
+---------+---------------------------------------------+-----------+---------+
2022-10-24 21:35:17,028 - distributed.worker - WARNING - Mismatched versions found

+---------+---------------------------------------------+-----------+---------+
| Package | Worker-1025ddc9-5761-4eda-8172-96db634378c7 | Scheduler | Workers |
+---------+------------------------------------------

21:35:26.858 | INFO    | Flow run 'vigilant-bee' - Finished in state Completed('All states completed.')
